In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# 如何给agent正确的增加记忆

- 将memory插入到提示词模板中
<hr>

In [12]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain
from langchain.prompts import  MessagesPlaceholder


定义大模型

In [13]:

llm=ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
    )

构建agent可用工具

In [14]:
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
#构建一个搜索工具
search = SerpAPIWrapper()
#创建一个数学计算工具
llm_math_chain = LLMMathChain(
    llm=llm,
    verbose=True
)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
print(tools)

[Tool(name='Search', description='useful for when you need to answer questions about current events or the current state of the world', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594', aiosession=None)>), Tool(name='Calculator', description='useful for when you need to answer questions about math', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


增加memory组件

In [15]:
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

定义agent

In [16]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    memory=memory #记忆组件
    )

查看默认的agents prompt是什么样的

In [17]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.'
prompt=PromptTemplate(input_variables=['input'], template='{input}')
variable_name='agent_scratchpad'


In [18]:
agent_chain.run("我叫什么名字？")



> Entering new AgentExecutor chain...
作为一个AI，我没有访问您个人信息的权限，所以我不知道您的名字。如果您愿意告诉我，我会知道如何称呼您。

> Finished chain.


'作为一个AI，我没有访问您个人信息的权限，所以我不知道您的名字。如果您愿意告诉我，我会知道如何称呼您。'

需要使用agent_kwargs传递参数，将chat_history传入

In [19]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    agent_kwargs={
        "extra_prompt_messages":[MessagesPlaceholder(variable_name="chat_history"),MessagesPlaceholder(variable_name="agent_scratchpad")],
    },
    memory=memory #记忆组件
    )

查看重新渲染后的提示词模版

In [20]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.'), MessagesPlaceholder(variable_name='chat_history'), MessagesPlaceholder(variable_name='agent_scratchpad'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.'
variable_name='chat_history'
variable_name='agent_scratchpad'


In [21]:
agent_chain.run("好厉害，刚才我们都聊了什么？")



> Entering new AgentExecutor chain...
作为一个文本交互的AI，我可以回顾我们的对话记录。到目前为止，我们的对话内容如下：

1. 您问我：“我叫什么名字？”
2. 我回答说，作为AI，我没有访问您个人信息的权限，所以我不知道您的名字，并提出如果您愿意告诉我，我会知道如何称呼您。

这就是我们目前为止的全部对话内容。如果您有其他问题或需要帮助，请随时告诉我！

> Finished chain.


'作为一个文本交互的AI，我可以回顾我们的对话记录。到目前为止，我们的对话内容如下：\n\n1. 您问我：“我叫什么名字？”\n2. 我回答说，作为AI，我没有访问您个人信息的权限，所以我不知道您的名字，并提出如果您愿意告诉我，我会知道如何称呼您。\n\n这就是我们目前为止的全部对话内容。如果您有其他问题或需要帮助，请随时告诉我！'